In [9]:
from torch.multiprocessing import Process, Queue, Lock
import os
import time, random
import torch

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.device = 'cuda:1'
        self.linear = torch.nn.Linear(3, 1)#.to(self.device)
        
    def forward(self, x):
        x = self.linear(x)
        return x.relu()
        
def producer(queue, lock, values):
    with lock:
        print(f'Starting Producer: {os.getpid()}')
        
    for value in values:
        time.sleep(random.randint(0, 10))
        rands = torch.randint(low=0, high=value, size=(value,)).float()
        mean = torch.mean(rands)
        sm = torch.sum(rands)
        queue.put(torch.tensor([value, mean, sm]))
    with lock:
        print(f'Producer {os.getpid()}, exiting')
        
def consumer(queue, lock, model):
    with lock:
        print(f'Starting Consumer {os.getpid()}')
    while True:
        #time.sleep(random.randint(0, 10))
        x = queue.get()
        x = x.to('cuda:1')
        #print('x', x)
        #x = x.to(self.device)
        #print(f'{os.getpid()} got {str(x)}')
        model = model.to('cuda:1')
        out = model(x)
        print('x', x, 'out', out)

        

In [3]:
s=set()
s.update([13,5,6 ,3,4 , 0])
s.update([2,4,6,7,10])
s

{0, 2, 3, 4, 5, 6, 7, 10, 13}

In [6]:
torch.tensor([5,7,5]).tolist()

[5, 7, 5]

In [10]:
if __name__=='__main__':
    if torch.multiprocessing.get_start_method() == 'fork':
        torch.multiprocessing.set_start_method('spawn', force=True)
    torch.multiprocessing.set_start_method('spawn', force=True)
    values = torch.randint(1, 100, size=(100, ))
    queue = Queue()
    lock = Lock()
    model = Model()
    producers = []
    consumers = []
    for i in range(4):
        producers.append(Process(target=producer, args=(queue, lock, values[i*25:(i+1)*25])))
    for j in range(1):
        p = Process(target=consumer, args=(queue, lock, model))
        p.daemon=True
        consumers.append(p)   
    
    for p in producers:
        p.start()
    
    for c in consumers:
        c.start()
        
    for p in producers:
        p.join()
        
    for c in consumers:
        c.join()
            
    print('Parent Process Exiting..')

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'producer' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'producer' on <module '__main__' (built-in)>
Traceback (most recent call last):

Parent Process Exiting..


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'consumer' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'producer' on <module '__main__' (built-in)>


In [11]:
import time
import os
import random
from multiprocessing import Process, Queue, Lock
 
 
# Producer function that places data on the Queue
def producer(queue, lock, names):
    # Synchronize access to the console
    with lock:
        print('Starting producer => {}'.format(os.getpid()))
         
    # Place our names on the Queue
    for name in names:
        time.sleep(random.randint(0, 10))
        queue.put(name)
 
    # Synchronize access to the console
    with lock:
        print('Producer {} exiting...'.format(os.getpid()))
 
 
# The consumer function takes data off of the Queue
def consumer(queue, lock):
    # Synchronize access to the console
    with lock:
        print('Starting consumer => {}'.format(os.getpid()))
     
    # Run indefinitely
    while True:
        time.sleep(random.randint(0, 10))
         
        # If the queue is empty, queue.get() will block until the queue has data
        name = queue.get()
 
        # Synchronize access to the console
        with lock:
            print('{} got {}'.format(os.getpid(), name))
 
 
if __name__ == '__main__':
     
    # Some lists with our favorite characters
    names = [['Master Shake', 'Meatwad', 'Frylock', 'Carl'],
             ['Early', 'Rusty', 'Sheriff', 'Granny', 'Lil'],
             ['Rick', 'Morty', 'Jerry', 'Summer', 'Beth']]
 
    # Create the Queue object
    queue = Queue()
     
    # Create a lock object to synchronize resource access
    lock = Lock()
 
    producers = []
    consumers = []
 
    for n in names:
        # Create our producer processes by passing the producer function and it's arguments
        producers.append(Process(target=producer, args=(queue, lock, n)))
 
    # Create consumer processes
    for i in range(len(names) * 2):
        p = Process(target=consumer, args=(queue, lock))
         
        # This is critical! The consumer function has an infinite loop
        # Which means it will never exit unless we set daemon to true
        p.daemon = True
        consumers.append(p)
 
    # Start the producers and consumer
    # The Python VM will launch new independent processes for each Process object
    for p in producers:
        p.start()
 
    for c in consumers:
        c.start()
 
    # Like threading, we have a join() method that synchronizes our program
    for p in producers:
        p.join()
 
    print('Parent process exiting...')

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'producer' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'producer' on <module '__main__' (built-in)>
Traceback (most recent call last):

Parent process exiting...


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'consumer' on <module '__main__' (built-in)>


In [12]:
torch.multiprocessing.set_start_method('fork', force=True)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'consumer' on <module '__main__' (built-in)>


In [13]:
model = Model()
model.forward_pro_con()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'consumer' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 105, in spawn_main
    exitcode = _main(fd)
  File "/home/yamanishi/.pyenv/versions/miniconda3-latest/envs/tr/lib/python3.7/multiprocessing/spawn.py", line 115, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'consumer' on <module '__main__' (built-in)>
Traceback (most recent call last):

AttributeError: 'Model' object has no attribute 'forward_pro_con'

In [ ]:
a = [torch.rand(10,10), torch.rand(20, 10), torch.rand(40,10)]
torch.cat(a, dim=1).size()

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 10 but got size 20 for tensor number 1 in the list.

In [ ]:
import random
num_calc=100
num_tasks=5
A=[0,5,4,5,2]
list(map(lambda _: (random.random(), A[_]), range(num_tasks)))

[(0.1462950125206013, 0),
 (0.9821005152771285, 5),
 (0.3021859665836796, 4),
 (0.10078272162970381, 5),
 (0.33966428175769503, 2)]

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import NeighborLoader

data = Planetoid('./', name='Cora')[0]

loader = NeighborLoader(
    data,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=[30] * 2,
    # Use a batch size of 128 for sampling training nodes
    batch_size=128,
    input_nodes=data.train_mask,
)

sampled_data = next(iter(loader))
print(sampled_data.batch_size)

128


In [ ]:
from tqdm import tqdm
train_file = '../data/cf/train.txt'
trainUniqueUsers, trainUser, trainItem, allPos = [], [], [], []
traindataSize, m_item, n_user = 0, 0, 0
with open(train_file) as f:
    for l in tqdm(f.readlines()):
        if len(l) > 0:
            l = l.strip('\n').split(' ')
            items = [int(i) for i in l[1:]]
            uid = int(l[0])
            trainUniqueUsers.append(uid)
            trainUser.extend([uid] * len(items))
            trainItem.extend(items)
            allPos.append(np.array(items))
            m_item = max(m_item, max(items))
            n_user = max(n_user, uid)
            traindataSize += len(items)

100%|██████████| 1104421/1104421 [00:10<00:00, 108886.43it/s]


In [ ]:
edge_index=torch.cat([torch.stack([torch.tensor(trainUser), torch.tensor(trainItem)+n_user], dim=0), torch.stack([torch.tensor(trainItem)+n_user, torch.tensor(trainUser)], dim=0)], dim=1)

In [ ]:
user = torch.randint(0, n_user, size=(20000,))

In [ ]:
from torch_geometric.loader import NeighborSampler
train_loader = NeighborSampler(edge_index, node_idx=user,
                               sizes=[10, 10], batch_size=2048,
                               shuffle=False, num_workers=12)

In [14]:
from torch_geometric.loader import NeighborSampler
train_loader = NeighborSampler(edge_index, node_idx=None,
                               sizes=[-1], batch_size=2048,
                               shuffle=False, num_workers=12)

In [15]:
for batch_size, n_id, adj in train_loader:
    edge_index, _, size = adj
    print(batch_size, n_id, adj, edge_index,size)

2048 tensor([   0,    1,    2,  ..., 8383, 8384, 8385]) EdgeIndex(edge_index=tensor([[1568, 1569, 1570,  ..., 7281, 7891, 8385],
        [   0,    0,    0,  ..., 1567, 1567, 1567]]), e_id=tensor([    0,     1,     2,  ..., 14337, 14338, 14339]), size=(8386, 2048)) tensor([[1568, 1569, 1570,  ..., 7281, 7891, 8385],
        [   0,    0,    0,  ..., 1567, 1567, 1567]]) (8386, 2048)
2048 tensor([2048, 2049, 2050,  ..., 4093, 4094, 4095]) EdgeIndex(edge_index=tensor([], size=(2, 0), dtype=torch.int64), e_id=tensor([], dtype=torch.int64), size=(2048, 2048)) tensor([], size=(2, 0), dtype=torch.int64) (2048, 2048)
2048 tensor([4096, 4097, 4098,  ..., 6141, 6142, 6143]) EdgeIndex(edge_index=tensor([], size=(2, 0), dtype=torch.int64), e_id=tensor([], dtype=torch.int64), size=(2048, 2048)) tensor([], size=(2, 0), dtype=torch.int64) (2048, 2048)
2048 tensor([6144, 6145, 6146,  ..., 8189, 8190, 8191]) EdgeIndex(edge_index=tensor([], size=(2, 0), dtype=torch.int64), e_id=tensor([], dtype=torch.int6

In [1]:
torch.arange(5, 11)

tensor([ 5,  6,  7,  8,  9, 10])

In [4]:
torch.cat([torch.zeros((5, 10))], dim=0)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [3]:
pos = torch.randint(n_user, n_user+m_item-1, size=(10000,))

In [4]:
from torch_geometric.loader import NeighborSampler
train_loader = NeighborSampler(edge_index, node_idx=pos,
                               sizes=[10, 10], batch_size=512,
                               shuffle=False, num_workers=12)

In [ ]:
from torch_geometric.loader import NegativeSampler


In [7]:
device = 'cpu'
node_idx = user
print(node_idx)
for batch_size, n_id, adjs in train_loader:
    # `adjs` holds a list of `(edge_index, e_id, size)` tuples.
    print('n_id',n_id)
    #for nn in n_id:
    #    print(nn)
    #print(pos)
    #print(n_id[:batch_size])
    print('n_id size', n_id.size())
    adjs = [adj.to(device) for adj in adjs]
    for i,(edge_index, _, size) in enumerate(adjs):
        print(edge_index)
        #for v in edge_index[1].unique():
        #    print(edge_index[:,edge_index[1]==v].size(1))
        print('edge index shape', edge_index.shape)
        #print(_)
        #print(_.size())
        print(size)

tensor([486302, 564387, 614087,  ..., 247308, 118692, 778755])
n_id tensor([   0,    1,    2,  ..., 8383, 8384, 8385])
n_id size torch.Size([8386])


AttributeError: 'tuple' object has no attribute 'to'

In [6]:
10000%512

272

In [1]:
a = set({1,2,3,4,5})
a.update(set([3,5,6,7]))
a

{1, 2, 3, 4, 5, 6, 7}